# Student Loan Risk with Deep Learning

In [2]:
# Imports
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from pathlib import Path

---

## Prepare the data to be used on a neural network model

### Step 1: Read the `student-loans.csv` file into a Pandas DataFrame. Review the DataFrame, looking for columns that could eventually define your features and target variables.   

In [3]:
# Read the csv into a Pandas DataFrame
file_path = "https://static.bc-edx.com/ai/ail-v-1-0/m18/lms/datasets/student-loans.csv"
loans_df = pd.read_csv(file_path)

# Review the DataFrame
loans_df.head()

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score,credit_ranking
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,0
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,0
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0


In [4]:
# Review the data types associated with the columns
loans_df.dtypes

payment_history           float64
location_parameter        float64
stem_degree_score         float64
gpa_ranking               float64
alumni_success            float64
study_major_code          float64
time_to_completion        float64
finance_workshop_score    float64
cohort_ranking            float64
total_loan_score          float64
financial_aid_score       float64
credit_ranking              int64
dtype: object

In [5]:
# Check the credit_ranking value counts
loans_df["credit_ranking"].value_counts()

credit_ranking
1    855
0    744
Name: count, dtype: int64

### Step 2: Using the preprocessed data, create the features (`X`) and target (`y`) datasets. The target dataset should be defined by the preprocessed DataFrame column “credit_ranking”. The remaining columns should define the features dataset.

In [6]:
# Define the target set y using the credit_ranking column
y = loans_df["credit_ranking"]

# Display a sample of y
y.head()

0    0
1    0
2    0
3    1
4    0
Name: credit_ranking, dtype: int64

In [7]:
# Define features set X by selecting all columns but credit_ranking
X = loans_df.drop(columns= "credit_ranking")

# Review the features DataFrame
X.head()

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4


### Step 3: Split the features and target sets into training and testing datasets.


In [8]:
# Split the preprocessed data into a training and testing dataset
# Assign the function a random_state equal to 1
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

### Step 4: Use scikit-learn's `StandardScaler` to scale the features data.

In [9]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the scaler to the features training dataset
scaler.fit(X_train)

# Scale the features
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

---

## Compile and Evaluate a Model Using a Neural Network

### Step 1: Create a deep neural network by assigning the number of input features, the number of layers, and the number of neurons on each layer using Tensorflow’s Keras.

> **Hint** You can start with a two-layer deep neural network model that uses the `relu` activation function for both layers.


In [10]:
# Define the the number of inputs (features) to the model
input_nodes = len(X.columns)

# Review the number of features
input_nodes

11

In [11]:
# Define the number of hidden nodes for the first hidden layer
hidden_first_nodes = 6

# Define the number of hidden nodes for the second hidden layer
hidden_second_nodes = 3

# Define the number of neurons in the output layer
output_neurons = 1

In [12]:
# Create the Sequential model instance
nn_model = tf.keras.models.Sequential()

# Add the first hidden layer
nn_model.add(tf.keras.layers.Dense(units=hidden_first_nodes, activation="relu", input_dim=input_nodes))

# Add the second hidden layer
nn_model.add(tf.keras.layers.Dense(units=hidden_second_nodes, activation="relu"))

# Add the output layer to the model specifying the number of output neurons and activation function
nn_model.add(tf.keras.layers.Dense(units=output_neurons, activation="sigmoid"))

In [13]:
# Display the Sequential model summary
nn_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 6)                 72        
                                                                 
 dense_1 (Dense)             (None, 3)                 21        
                                                                 
 dense_2 (Dense)             (None, 1)                 4         
                                                                 
Total params: 97 (388.00 Byte)
Trainable params: 97 (388.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


### Step 2: Compile and fit the model using the `binary_crossentropy` loss function, the `adam` optimizer, and the `accuracy` evaluation metric.


In [14]:
# Compile the Sequential model
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [15]:
# Fit the model using 50 epochs and the training data
nn_model.fit(X_train_scaled, y_train, epochs=50)

Epoch 1/50
38/38 [==============================] - 2s 4ms/step - loss: 0.8499 - accuracy: 0.3720
Epoch 2/50
38/38 [==============================] - 0s 3ms/step - loss: 0.7710 - accuracy: 0.3995
Epoch 3/50
38/38 [==============================] - 0s 3ms/step - loss: 0.7226 - accuracy: 0.4454
Epoch 4/50
38/38 [==============================] - 0s 3ms/step - loss: 0.6948 - accuracy: 0.4937
Epoch 5/50
38/38 [==============================] - 0s 3ms/step - loss: 0.6771 - accuracy: 0.5338
Epoch 6/50
38/38 [==============================] - 0s 3ms/step - loss: 0.6643 - accuracy: 0.5488
Epoch 7/50
38/38 [==============================] - 0s 4ms/step - loss: 0.6533 - accuracy: 0.5571
Epoch 8/50
38/38 [==============================] - 0s 3ms/step - loss: 0.6423 - accuracy: 0.5621
Epoch 9/50
38/38 [==============================] - 0s 3ms/step - loss: 0.6298 - accuracy: 0.5613
Epoch 10/50
38/38 [==============================] - 0s 3ms/step - loss: 0.6167 - accuracy: 0.5630
Epoch 11/50
38/38 [

### Step 3: Evaluate the model using the test data to determine the model’s loss and accuracy.


In [16]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled, y_test, verbose=2)

# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

13/13 - 0s - loss: 0.5391 - accuracy: 0.7350 - 198ms/epoch - 15ms/step
Loss: 0.53905189037323, Accuracy: 0.7350000143051147


### Step 4: Save and export your model to a keras file, and name the file `student_loans.keras`.


In [18]:
# Set the model's file path
file_path = Path("student_loans.keras")

# Export your model to a keras file
nn_model.save(file_path)

---
## Predict Loan Repayment Success by Using your Neural Network Model

### Step 1: Reload your saved model.

In [19]:
# Set the model's file path
file_path = Path("student_loans.keras")

# Load the model to a new object
nn_imported_model = tf.keras.models.load_model(file_path)

### Step 2: Make predictions on the testing data and save the predictions to a DataFrame.

In [22]:
# Make predictions with the test data
nn_predictions = nn_imported_model.predict(X_test,verbose=2)

# Display a sample of the predictions
nn_predictions

13/13 - 0s - 168ms/epoch - 13ms/step


array([[1.03447428e-08],
       [5.28078032e-15],
       [4.58114743e-01],
       [3.50174112e-10],
       [1.64789870e-01],
       [2.15279907e-01],
       [3.88950016e-03],
       [1.08154575e-24],
       [9.31091130e-01],
       [6.09723330e-01],
       [2.73602252e-09],
       [4.00668441e-08],
       [3.07631965e-14],
       [6.35894714e-03],
       [5.33094957e-10],
       [6.20950028e-11],
       [2.97857991e-06],
       [3.95806646e-03],
       [2.89566349e-02],
       [9.16640238e-06],
       [2.88477540e-01],
       [1.27107391e-08],
       [1.68766128e-05],
       [9.99984682e-01],
       [5.08296572e-18],
       [1.36920255e-06],
       [5.99334180e-01],
       [1.06221930e-14],
       [8.79635600e-11],
       [0.00000000e+00],
       [2.05548466e-04],
       [9.99323070e-01],
       [3.72894649e-19],
       [5.74161947e-01],
       [1.00061234e-19],
       [7.64573734e-13],
       [2.24498101e-02],
       [9.98347253e-02],
       [9.99032199e-01],
       [9.07053892e-03],


In [24]:
# Save the predictions to a DataFrame and round the predictions to binary results
nn_predictions_rounded = [round(i[0],0) for i in nn_predictions]
nn_predictions_df = pd.DataFrame(nn_predictions_rounded, columns=["predictions"])
nn_predictions_df

,predictions
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0
...,...
395,0.0
396,0.0
397,0.0
398,0.0


### Step 4: Display a classification report with the y test data and predictions

In [26]:
# Print the classification report with the y test data and predictions
print(classification_report(y_test, nn_predictions_rounded))

              precision    recall  f1-score   support

           0       0.48      0.88      0.62       188
           1       0.58      0.15      0.23       212

    accuracy                           0.49       400
   macro avg       0.53      0.51      0.43       400
weighted avg       0.53      0.49      0.42       400



In [29]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   payment_history         1599 non-null   float64
 1   location_parameter      1599 non-null   float64
 2   stem_degree_score       1599 non-null   float64
 3   gpa_ranking             1599 non-null   float64
 4   alumni_success          1599 non-null   float64
 5   study_major_code        1599 non-null   float64
 6   time_to_completion      1599 non-null   float64
 7   finance_workshop_score  1599 non-null   float64
 8   cohort_ranking          1599 non-null   float64
 9   total_loan_score        1599 non-null   float64
 10  financial_aid_score     1599 non-null   float64
dtypes: float64(11)
memory usage: 137.5 KB


---
## Discuss creating a recommendation system for student loans

Briefly answer the following questions in the space provided:

1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.

2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.

3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.

**1. First, I would need additional demographic data from the customer, such as age (or date of birth), gender, employment history, current net worth, current income, etc. Second, I would need a lot more information about other students' (current and historical) loan terms (e.g., loan type, loan amount, interest rate, loan term, collateral, repayment schedule, whether the loan was paid off in full, etc.), along with macroeconomic data during those times.**


**2. My model would use a mix of collaborative filtering and context-based filtering. Content-based filtering would be less useful given that most students do not obtain that many loans and have almost no credit history, therefore it would be difficult to use that small sample size to make recommendations on future loans.**

**The data from other students (e.g., the loan terms) would be useful for collaborative filtering because I could match a similar profile to the applicant to determine what kind of loan they should be provided. The additional demographic data (and the data already provided in the student-loans csv) would be useful for context-based filtering because I could determine the likelihood of repayment based on similar factors from the data.**


**3. Two real-world challenges are A) unintended bias, and B) data quality. Unintended bias would be a concern because the whole intention of lending is to make both parties better off. If one party continues to be rejected for things out of their control, then that would leave a big hole in the sought-out goals of lending. Data quality would be a concern because the data that the credit decision is based on may be lacking or incorrect, therefore making erroneous recommendations.**

